In [1]:
import torch
from torch import nn

## Construct the model and initialization

In [6]:
num_inputs, num_outputs, num_hiddens = 28 * 28, 10, 256

W1 = nn.Parameter(
    torch.randn(num_inputs, num_hiddens, requires_grad=True) * 0.01
)
b1 = nn.Parameter(
    torch.zeros(num_hiddens, requires_grad=True)
)

W2 = nn.Parameter(
    torch.randn(num_hiddens, num_outputs, requires_grad=True) * 0.01
)
b2 = nn.Parameter(
    torch.zeros(num_outputs, requires_grad=True)
)

params = [W1, b1, W2, b2]

In [7]:
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(a, X)

In [9]:
def net(X):
    X = X.reshape(-1, num_inputs)
    print(X.shape)
    H = relu(X@W1 + b1)
    O = H@W2 + b2
    return O

In [10]:
loss = nn.CrossEntropyLoss(reduction='none')

## Train

In [ ]:
num_epochs, lr = 10, 0.01

updater = torch.optim.SGD(params, lr=lr)

In [ ]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)

    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [ ]:
def train_epoch(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):
        net.train()
        
    metric = Accumulator(3)
    
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            l.sum().backward()
            updater(X.shape[0])

        metric.add(float(l.sum()), accuracy(y_hat, y))